In [ ]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [ ]:
import logging
import delta_sharing
import pandas as pd

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [ ]:
customer = "Rosental"
fb_param="adb_ad_id"
start_date = "20250308"
end_date = "20250508"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [ ]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.{account_id}.features_view_30_outlook"
df = delta_sharing.load_as_pandas(table_path, limit=1000)

In [26]:
# query sessions
sessions=send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"sessionId":df["session"].dropna().unique().tolist()},
    logging
)
sessions = pd.json_normalize(sessions)
sessions.columns

In [27]:
sessions[["created","sessionId","campaign.gclid",f"campaign.{fb_param}"]]

In [28]:
sessions[sessions["campaign.gclid"].isnull()==False]

In [29]:
gclids = sessions["campaign.gclid"].dropna().unique().tolist()
print(f"gclids = {len(gclids)}")

In [31]:
signals_gclids=send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":gclids},
    logging
)
signals_gclids = pd.json_normalize(signals_gclids)
signals_gclids.shape

In [32]:
fb = sessions[f"campaign.{fb_param}"].dropna().unique().tolist()
print(len(fb))

In [38]:
signals_fb_response=send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":fb},
    logging
)
signals_fb = pd.json_normalize(signals_fb_response)
signals_fb.shape

In [36]:
signals_fb_response

In [40]:
print(len(signals_fb_response))
signals_fb_response += signals_fb_response
print(len(signals_fb_response))


In [34]:
signals_fb[["externalId","relates_to.treatment"]]